# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [ ]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import numpy as np
import torch
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models
from collections import OrderedDict
from PIL import Image
import glob, os

In [ ]:
# CONSTANTS
ROTATION_AMOUNT = 30
CROP_AMOUNT = 224
RESIZE_AMOUNT = 225
COLOR_CHANNEL_1 = 0.485, 0.456, 0.406
COLOR_CHANNEL_2 = 0.229, 0.224, 0.225
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EPOCHS = 1
STEPS = 0
PRINT_EVERY = 5
PIL_SIZE = 256

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [ ]:
data_dir = 'assets/flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# TODO: Define your transforms for the training, validation, and testing sets

train_transforms = transforms.Compose([transforms.RandomRotation(ROTATION_AMOUNT),
                                       transforms.RandomResizedCrop(CROP_AMOUNT), # model takes 224x224 images as input, so we resize all of them
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize(COLOR_CHANNEL_1,
                                       COLOR_CHANNEL_2)])

valid_transforms = transforms.Compose([transforms.Resize(RESIZE_AMOUNT),
                                      transforms.CenterCrop(CROP_AMOUNT),
                                      transforms.ToTensor(),
                                      transforms.Normalize(COLOR_CHANNEL_1,
                                      COLOR_CHANNEL_2)])                                

test_transforms = transforms.Compose([transforms.Resize(RESIZE_AMOUNT),
                                      transforms.CenterCrop(CROP_AMOUNT),
                                      transforms.ToTensor(),
                                      transforms.Normalize(COLOR_CHANNEL_1,
                                      COLOR_CHANNEL_2)])  



# TODO: Load the datasets with ImageFolder

# Pass transforms in here, then run the next cell to see how the transforms look
image_train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
image_valid_dataset = datasets.ImageFolder(valid_dir, transform=valid_transforms)
image_test_dataset = datasets.ImageFolder(test_dir, transform=test_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders

train_dataloader = torch.utils.data.DataLoader(image_train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(image_valid_dataset, batch_size=BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(image_test_dataset, batch_size=BATCH_SIZE)

In [ ]:
# look at the shape and size of these images, they are (64, 3, 224, 224)
dataiter = iter(train_dataloader)
check_image, check_label = dataiter.next()
print(type(check_image))
print(check_image.shape)

In [ ]:
# here is what one of those images looks like
plt.imshow(np.transpose(check_image[1], (1, 2, 0)))

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [ ]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
    cat_count = len(cat_to_name.keys())
    print(cat_count)
    print(cat_to_name)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [ ]:
# determine available architecture
this_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# choose a model 
model = models.resnet50(pretrained=True)
model

In [ ]:
# check model object - see in and out features etc
print(model.fc) 

In [ ]:
# TODO: Build and train your network

# Freeze our feature paramaters - turn of gradients for our model
for param in model.parameters():
    param.requires_grad = False

# Define our new classifier
classifier = nn.Sequential(
                      nn.Linear(in_features=2048, out_features=cat_count),
                      nn.LogSoftmax(dim=1)
)


# replace current models classifier with this one
model.fc = classifier


In [ ]:
# check model object - see in and out features etc
print(model.fc) 

In [ ]:
# specify loss function (categorical cross-entropy)
criterion = nn.NLLLoss()

# specify optimizer (stochastic gradient descent) and use the default learning rate (0.001)
optimizer = optim.Adam(model.fc.parameters(), lr=LEARNING_RATE)

# model to gpu if this device suports
model.to(this_device);

In [ ]:
# Train and Validate the Model
epochs = EPOCHS
steps = STEPS
print_every = PRINT_EVERY

running_loss = 0
# NB// by default, the model is set to "training" mode
for epoch in range(epochs):
    for inputs, labels in train_dataloader:
        steps += 1
        # move the input and label tensors to this device (cpu or gpu if available)
        inputs, labels = inputs.to(this_device), labels.to(this_device)
        
        # get log probabilities from our model and pass in images
        logps = model.forward(inputs)

        # calculate the batch loss
        loss = criterion(logps, labels)
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()

        # do a backwards pass
        loss.backward()

        # perform a single optimization step (parameter update)
        optimizer.step()

        # keep track of training loss
        running_loss += loss.item()
        
        if steps % print_every == 0:
            # Keep track of training and validation loss
            validation_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in valid_dataloader:
                    inputs, labels = inputs.to(this_device), labels.to(this_device)
                    logps = model.forward(inputs)
                    
                    # update loss metrics
                    batch_loss = criterion(logps, labels)
                    validation_loss += batch_loss.item()
                    
                     # calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Validation loss: {validation_loss/len(valid_dataloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(valid_dataloader):.3f}")
            running_loss = 0
            model.train()

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
#TODO: Do validation on the test set
# eval mode
epochs = EPOCHS
model.eval()
test_accuracy = 0
test_loss = 0
# iterate over epochs
for epoch in range(epochs):
    with torch.no_grad():

        # iterate over test data
        for images, labels in test_dataloader:

            #send to gpu, if available
            images, labels = images.to(this_device),labels.to(this_device)

            # forward pass: compute predicted outputs by passing inputs to the model
            output = model.forward(images)

            # update test loss 
            test_loss += criterion(output,labels).item()
            
            # convert output probabilities to predicted class
            predicted_class = torch.exp(output)

            #match the test
            test_match = (labels.data == predicted_class.max(dim=1)[1])
            
            # calculate the test accuracy
            test_accuracy += test_match.type(torch.FloatTensor).mean()
            
            #print the test output
            print("Epoch: {}/{}.. ".format(epoch+1, epochs),
            "Test loss: {:.3f}.. ".format(test_loss/len(test_dataloader)),
            "Test accuracy: {:.3f}".format(test_accuracy/len(test_dataloader)))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# the parameters for PyTorch networks are stored in a model's `state_dict`.
print("Our model: \n\n", model, '\n')
print("The state dict keys: \n\n", model.state_dict().keys())

In [ ]:
print(image_train_dataset.class_to_idx)

In [ ]:
torch.save({
            'epoch': EPOCHS,
            'fc': model.fc,
            'class_to_idx': image_train_dataset.class_to_idx,
            'optimizer_state_dict': optimizer.state_dict,
            'state_dict': model.state_dict()
            }, 'checkpoint.pth')


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint_model(path):
    # load the checkpoint
    checkpoint = torch.load(path)

    # load the model
    checkpoint_model = models.resnet50(pretrained=True)

    # freeze the parameters
    for param in checkpoint_model.parameters():
        param.requires_grad = False

    # load up the optimiser
    checkpoint_model.optimizer = checkpoint['optimizer_state_dict']

    # apply custom fc back to the model
    checkpoint_model.fc = checkpoint['fc']
    checkpoint_model.load_state_dict(checkpoint['state_dict'])
    checkpoint_model.class_to_idx = checkpoint['class_to_idx']

    # run eval mode
    checkpoint_model.eval()

    return checkpoint_model

In [ ]:
path = 'checkpoint.pth'
checkpoint_model = load_checkpoint_model(path)

In [ ]:
print(checkpoint_model.optimizer)

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    # TODO: Process a PIL image for use in a PyTorch model
    # resize where the sortest is 256, maintain aspect ratio
    size = PIL_SIZE, PIL_SIZE

    # open the image
    im  = Image.open(image)

    #use PIL thumbnail func to resize
    im.thumbnail(size)

    # retrieve image width and height
    im_width, im_height = im.size

    # set crop vars to crop out the centre
    crop_width, crop_height = CROP_AMOUNT,CROP_AMOUNT

    # work out crops
    left = (im_width - crop_width) / 2
    top = (im_height - crop_height) / 2
    right = (im_width + crop_width) / 2
    bottom = (im_height + crop_height) / 2

    #apply crops
    im =im.crop((left, top, right, bottom))
    
    # work our numpy colour channels
    np_image = np.array(im)
    np_image = np_image / 255

    #set up numpy colour channels as previous
    std = np.array(COLOR_CHANNEL_1)
    mean = np.array(COLOR_CHANNEL_2)

    # apply numpy colour channel calcs
    np_image = (np_image - mean) / std

    # apply to image
    image = np.transpose(np_image,(2,0,1))

    #return
    return torch.from_numpy(image)

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [ ]:
# Test the process image function
# load up an image
image = process_image('assets/flower_data/test/7/image_07211.jpg')
imshow(image)

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    # run the modal in eval mode
    model.eval()

    # fix unexpected scalar type Double but found Float
    model.double()

    # model to gpu if this device suports
    model = model.to(this_device);
    
    # call our process_image func above
    image = process_image(image_path)
    
    # image to this device - unsqueeze the tensor
    image = image.to(this_device).unsqueeze(0)

    with torch.no_grad():
        # extract the top-k value-indices
        output = model.forward(image)
        topk ,topk_labels = torch.topk(output, topk)
        topk = topk.exp()

    # pupulate idx_to_class for labels
    idx_to_class = {model.class_to_idx[k]: k for k in model.class_to_idx}

    # create an empty array to store the class labels in
    classes = []

    # loop though predictions and populate classes
    for label in topk_labels.numpy()[0]:
        classes.append(idx_to_class[label])

    # retun topk and classes array
    return topk.numpy()[0], classes

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

# grab an image to test with
test_image = 'assets/flower_data/test/92/image_03032.jpg'

# call the predict function
topk, top_classes = predict(test_image, checkpoint_model)

# preprocess the image with the image preprossessing function
processed_image = process_image(test_image)

# find the maximum index
maximum_index = top_classes[0]

# set up our figure
figure = plt.figure(figsize =[10,5])

# set up axis properties
axis = imshow(processed_image, ax = plt)
axis.axis('off')
axis.title(cat_to_name[str(maximum_index)])
axis.show()

# empty array to store labels in
labels = []

# loop through each class in our top classes and populate labels
for each_class in top_classes:
    labels.append(cat_to_name[each_class])

# set up axis properties
axis2= plt.subplot(111)
yticks = np.arange(5)
axis2.set_yticks(yticks)
axis2.set_yticklabels(labels)
axis2.barh(yticks, topk, xerr=0, align='center', color='blue')

# invert the y axis
axis2.invert_yaxis()

plt.show()